# **Salary Insights: Predicting Data Analytics Salaries**


<h1>Glassdoor Jobs Dataset</h1>

| Column Name | Description |
|---|---|
| Job Title | The specific job position advertised. |
| Salary Estimate | The estimated salary range for the position, including minimum and maximum values, as well as hourly or annual rates. |
| Job Description | A detailed description of the job responsibilities, qualifications, and requirements. |
| Rating | The rating given to the company by its employees (1 --> 5). |
| Company Name | The name of the company posting the job. |
| Location | The city, state, and country where the job is located. |
| Headquarters | The location of the company's headquarters. |
| Size | The estimated size of the company (e.g., estimated range of Number of employees). |
| Founded | The year the company was founded. |
| Type of ownership | The type of ownership (e.g., public, private, government). |
| Industry | The industry sector in which the company operates (e.g., technology, healthcare, finance). |
| Sector | A broader category that groups related industries (e.g., technology, healthcare, manufacturing). |
| Revenue | estimated range of company revenue. |
| Competitors | A list of the company's main competitors. |

In [1]:
#Importing libraries
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np 
import re

In [2]:
#Reading the data
df = pd.read_csv('glassdoor_jobs.csv')
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [3]:
df.shape

(956, 15)

In [4]:
# Remove rows where 'Salary Estimate' is -1, indicating missing or invalid data
df = df[df['Salary Estimate'] != '-1']

In [5]:
df.shape

(742, 15)

In [6]:
#Checking Null values
df.isnull().sum()

Unnamed: 0           0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
dtype: int64

In [7]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [8]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
df['Job Title'].value_counts()

Job Title
Data Scientist                     131
Data Engineer                       53
Senior Data Scientist               34
Data Analyst                        15
Senior Data Engineer                14
                                  ... 
Data Scientist/ML Engineer           1
Sr. Data Scientist                   1
Data Engineer 4 - Contract           1
Data Analyst - Asset Management      1
Machine Learning Engineer (NLP)      1
Name: count, Length: 264, dtype: int64

In [10]:
def title_simplifier(title):
  """Simplifies job titles by mapping common job titles to standardized terms.

  Args:
    title: The job title to simplify.

  Returns:
    The simplified job title, or 'na' if no match is found.
  """

  title = title.lower()

  title_mapping = {
      'data scientist': 'Data Scientist',
      'data engineer': 'Data Engineer',
      'analyst': 'Data Analyst',
      'machine learning': 'MLE',
      'manager': 'Manager',
      'director': 'Director'
  }

  for key in title_mapping:
      if key in title:
          return title_mapping[key]

  return 'na'

df['job_simp'] = df['Job Title'].apply(title_simplifier)

In [11]:
df['job_simp'].value_counts()

job_simp
Data Scientist    279
na                184
Data Engineer     119
Data Analyst      102
Manager            22
MLE                22
Director           14
Name: count, dtype: int64

In [13]:
df['Salary Estimate']

0       $53K-$91K (Glassdoor est.)
1      $63K-$112K (Glassdoor est.)
2       $80K-$90K (Glassdoor est.)
3       $56K-$97K (Glassdoor est.)
4      $86K-$143K (Glassdoor est.)
                  ...             
950    $58K-$111K (Glassdoor est.)
951    $72K-$133K (Glassdoor est.)
952     $56K-$91K (Glassdoor est.)
953    $95K-$160K (Glassdoor est.)
955    $61K-$126K (Glassdoor est.)
Name: Salary Estimate, Length: 742, dtype: object

In [14]:
# Removing (Glassdoor est.) from the column
df['Salary'] = df['Salary Estimate'].apply(lambda x: x.split('(')[0])

# Removing '$' and 'K' from the column
df['Salary'] = df['Salary'].str.replace('$', '').str.replace('K','')

In [15]:
df['Salary'].isnull().sum()

0

In [16]:
#Checking if the salary is hourly paid or not, as it differs in average Salary
df['Hourly_Pay'] = df['Salary'].apply(lambda x: 1 if 'per hour' in x.lower() else 0) 
#Checking if estimated salary is provided by an employer or not
df['Employer_Provided'] = df['Salary'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

In [17]:
df['Salary'] = df['Salary'].str.replace('per hour', '', case=False)
df['Salary'] = df['Salary'].str.replace('employer provided salary:', '', case=False)

In [18]:
#Extracting Average Salary Column from Salary column
df['Maximum Salary'] = pd.to_numeric(df['Salary'].apply(lambda x: x.split('-')[1]))
df['Minimum Salary'] = pd.to_numeric(df['Salary'].apply(lambda x: x.split('-')[0]))
df['Average Salary'] = (df['Minimum Salary'] + df['Maximum Salary']) / 2

In [19]:
df['Minimum Salary'] = df.apply(lambda x: x['Minimum Salary'] * 2 if x['Hourly_Pay'] == 1 else x['Minimum Salary'], axis = 1 )
df['Maximum Salary'] = df.apply(lambda x: x['Maximum Salary'] * 2 if x['Hourly_Pay'] == 1 else x['Maximum Salary'], axis = 1 )

In [20]:
df['Average Salary'].isnull().sum()

0

In [21]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'job_simp', 'Salary', 'Hourly_Pay', 'Employer_Provided',
       'Maximum Salary', 'Minimum Salary', 'Average Salary'],
      dtype='object')

In [22]:
top_data_science_tools = [
    "Python","Scala", "Java","SQL",
    "Scikit-learn", "TensorFlow", "Keras",
    "Matplotlib", "Tableau", "PowerBI", "Plotly", "D3.js",
    "Apache Spark", "Hadoop",
    "AWS", "Google Cloud",
    "Pandas", "NumPy","Excel"
]

In [23]:
for tool in top_data_science_tools:
    df[tool.lower()] = df['Job Description'].apply(lambda x: 1 if tool.lower() in x.lower() else 0)
    

In [24]:
import re 
# Create a regular expression pattern from the list of skills
skills_pattern = r'\b(' + '|'.join(top_data_science_tools) + r')\b'

# Function to extract skills
def extract_skills(job_description):
    skills = re.findall(skills_pattern, job_description)
    return list(set(skills))  # Return unique skills

# Apply the function to the Job Description column
df['Extracted Skills'] = df['Job Description'].apply(extract_skills)

In [26]:
df[['Job Description','Extracted Skills']]

,Job Description,Extracted Skills
0,"Data Scientist\nLocation: Albuquerque, NM\nEdu...","[Excel, Tableau, Python]"
1,What You Will Do:\n\nI. General Summary\n\nThe...,"[Java, Python]"
2,"KnowBe4, Inc. is a high growth information sec...","[Excel, SQL, Python, D3.js]"
3,*Organization and Job ID**\nJob ID: 310709\n\n...,[Python]
4,Data Scientist\nAffinity Solutions / Marketing...,"[SQL, Python]"
...,...,...
950,Site Name: USA - Massachusetts - Cambridge\nPo...,[]
951,THE CHALLENGE\nEventbrite has a world-class da...,"[AWS, Java, Scala, Python, SQL, Hadoop]"
952,The Auton Lab at Carnegie Mellon University is...,[]
953,Data Science ManagerResponsibilities:\n\nOvers...,[]


In [27]:
#Checking the Educational requirements in every job description
education_requirements = [
    "Master's Degree",
    "PhD",
    "Bachelor's Degree"
]
def check_education_requirements(job_description):
    found_requirements = [req for req in education_requirements if req.lower() in job_description.lower()]
    return ', '.join(found_requirements) if found_requirements else None


df['Educational Requirements'] = df['Job Description'].apply(check_education_requirements)
df['Educational Requirements'].value_counts()

Educational Requirements
PhD                                        156
Bachelor's Degree                           83
Master's Degree                             25
Master's Degree, Bachelor's Degree          16
Master's Degree, PhD                        10
Master's Degree, PhD, Bachelor's Degree      3
Name: count, dtype: int64

In [28]:
df['Company'] = df['Company Name'].apply(lambda x:x.split('\n')[0])

In [29]:
df['State'] = df['Location'].apply(lambda x: x.split(',')[-1])

In [30]:
df['Same State'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis=1)

In [31]:
df['Size'].value_counts().index

Index(['1001 to 5000 employees', '501 to 1000 employees', '10000+ employees',
       '201 to 500 employees', '51 to 200 employees',
       '5001 to 10000 employees', '1 to 50 employees', 'Unknown', '-1'],
      dtype='object', name='Size')

In [32]:
"""
Classifying the company based on the number of employees, as there is 
a positive correlation exists between company size and employee salaries, indicating that larger companies tend to offer higher compensation.
"""
def classify_company_size(size):
    if size in ['Unknown', '-1']:
        return 'Unknown'
    
    # Extract numeric values using regex
    match = re.findall(r'(\d+)', size)
    
    if match:
        # Calculate the average size if there are two numbers
        if len(match) == 2:
            size_value = (int(match[0]) + int(match[1])) / 2
        else:
            size_value = int(match[0])  # For cases like '10000+ employees'
            
            # Classify '10000+' as Enterprise
            if size_value >= 10000:
                return "Enterprise"
    else:
        return 'Unknown'
    
    # Classify based on the extracted size value
    if size_value <= 50:
        return "Micro"
    elif size_value <= 200:
        return "Small"
    elif size_value <= 1000:
        return "Medium"
    elif size_value <= 5000:
        return "Large"
    elif size_value <= 10000:
        return "Very Large"
    
    return "Enterprise"  # Fallback for any larger sizes

# Apply the classification function to the Company Size column
df['Classified Size'] = df['Size'].apply(classify_company_size)


In [43]:
df['Classified Size'].value_counts()

Classified Size
Medium        251
Large         150
Enterprise    130
Small          94
Very Large     76
Micro          31
Unknown        10
Name: count, dtype: int64

In [33]:
df['Size'].value_counts()

Size
1001 to 5000 employees     150
501 to 1000 employees      134
10000+ employees           130
201 to 500 employees       117
51 to 200 employees         94
5001 to 10000 employees     76
1 to 50 employees           31
Unknown                      9
-1                           1
Name: count, dtype: int64

In [34]:
#Extracting the Age of company from Founded column, This will be more useful.
df['Age_of_company'] = df.Founded.apply(lambda x : x if x < 1 else 2024 - x)

In [35]:
#Extracting the number of Competitors for each company
df['Num_Competitors'] = df['Competitors'].apply(lambda x: len(x.split(',')) if x != -1 else 0)

In [36]:
df['Num_Competitors'].value_counts()

Num_Competitors
1    472
3    228
2     41
4      1
Name: count, dtype: int64

In [37]:
# Function to convert revenue to numeric value
def convert_revenue(revenue_str):
    # Regular expression to extract the numeric values
    match = re.findall(r'\$(\d+)(?: to \$(\d+))? million', revenue_str)
    if match:
        # Extract lower and upper bounds
        lower, upper = match[0]
        lower_value = float(lower) * 1_000_000  # Convert to numeric
        if upper:  # If there is an upper bound
            upper_value = float(upper) * 1_000_000  # Convert to numeric
            return (lower_value + upper_value) / 2  # Return average
        return lower_value  # Return only lower bound if no upper bound
    return np.nan  # Return NaN for unrecognized formats

# Apply the function to the Revenue column
df['Revenue Numeric'] = df['Revenue'].apply(convert_revenue)

In [39]:
df['Type of ownership'] = df['Type of ownership'].str.replace("-1", "Unknown")

In [40]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'job_simp', 'Salary', 'Hourly_Pay', 'Employer_Provided',
       'Maximum Salary', 'Minimum Salary', 'Average Salary', 'python', 'scala',
       'java', 'sql', 'scikit-learn', 'tensorflow', 'keras', 'matplotlib',
       'tableau', 'powerbi', 'plotly', 'd3.js', 'apache spark', 'hadoop',
       'aws', 'google cloud', 'pandas', 'numpy', 'excel', 'Extracted Skills',
       'Educational Requirements', 'Company', 'State', 'Same State',
       'Classified Size', 'Age_of_company', 'Num_Competitors',
       'Revenue Numeric'],
      dtype='object')

In [41]:
df.to_csv('Cleaned_Data.csv', index=False)